In [25]:
data Ty
  = Int         -- int
  | Arr Ty Int  -- ty[3]
  | Ptr Ty      -- ty*
  | Str String  -- struct list
  deriving Show

data SDec = SDec String [VDec]     -- struct list { ... };
  deriving Show

data VDec = VDec Ty String         -- int x;
  deriving Show

data FDec = FDec Ty String [VDec] [VDec] Stmt
  deriving Show

data Stmt
  = Sexp Exp                  --  e;
  | Sseq [Stmt]               --  { ss }  where ss is either empty or e1; e2; ... 
  | If Exp Stmt               --  if (e) s
  | IfElse Exp Stmt Stmt      --  if (e) s1 else s2
  -- | Switch Exp [Stmt] Stmt    --  switch (e) { case 0: ss1 break; case 1: ss0 break; ... default: ssk }
  | While Exp Stmt            --  while (e) s
  | For (Exp, Exp, Exp) Stmt  --  for (e1; e2; e3) s
  deriving Show
  
data Exp 
  = Lit Int            --  3
  | Var String         --  x
  | And Exp Exp        --  e1 && e2
  | Or Exp Exp         --  e1 || e2
  | Not Exp            --  ! e
  | Equ Exp Exp        --  e1 == e2
  | Neq Exp Exp        --  e1 != e2
  | Gr  Exp Exp        --  e1 > e2
  | Geq Exp Exp        --  e1 >= e2
  | Le  Exp Exp        --  e1 < e2
  | Leq Exp Exp        --  e1 <= e2
  | Neg Exp            --  - e
  | Add Exp Exp        --  e1 + e2
  | Sub Exp Exp        --  e1 - e2
  | Mul Exp Exp        --  e1 * e2
  | Div Exp Exp        --  e1 / e2
  | Assign Exp Exp     --  e1 <- e2
  | Sizeof Ty          --  sizeof(ty)
  | Ref                --  & e
  | Deref              --  * e
  | Func String [Exp]  -- f(e1, e2, ...)
  deriving Show

data Prog = Prog [SDec] [VDec] [FDec] Stmt  deriving Show

In [26]:
import Data.List (intersperse)

type Code = [Inst]

type Addr = Int

data Inst
  = ADD | SUB | MUL | DIV | NEG
  | AND | OR | NOT
  | EQU | NEQ | GR | GEQ | LE | LEQ
  | DUP | POP
  | JUMP Code | JUMPz Code   --  | JUMPi [Code]
  | LOAD | LOADc Int    --  | LOADa | LOADr | LOADrc
  | STORE | STOREc Int  --  | STOREa | STOREr
--  | ALLOC | SLIDE | CALL | ENTER  | RETURN
--  | MALLOC | NEW
--  | MARK
  | HALT
  deriving (Eq, Ord)

type AEnv = [(String, Addr)]
type TEnv = [(String, Ty)]
type SEnv = [(String, [TEnv])]

In [27]:
instance Show Inst where
  show = showInst 0

-- cyclic data가 무한히 출력되는 것을 막기 위해 깊이 제한
showInst k ADD = "ADD"
showInst k SUB = "SUB"
showInst k MUL = "MUL"
showInst k DIV = "DIV"
showInst k NEG = "NEG"
showInst k AND = "AND"
showInst k OR  = "OR"
showInst k NOT = "NOT"
showInst k EQU = "EQU"
showInst k NEQ = "NEQ"
showInst k GR  = "GR"
showInst k GEQ = "GEQ"
showInst k LE  = "LE"
showInst k LEQ = "LEQ"
showInst k DUP = "DUP"
showInst k POP = "POP"
showInst k (JUMP c)  | k <= 0     = "JUMP "++"<code>"
                     | otherwise = "JUMP ["++(concat . intersperse ", " . map (showInst (k-1)) $ c)++"]"
showInst k (JUMPz c) | k <= 0     = "JUMPz "++"<code>"
                     | otherwise = "JUMPz ["++(concat . intersperse ", " . map (showInst (k-1)) $ c)++"]"
--  | JUMPi [Code]
showInst k LOAD = "LOAD"
showInst k (LOADc n) = "LOADc "++show n
--  | LOADa | LOADr | LOADrc
showInst k STORE = "STORE"
showInst k (STOREc n) = "STOREc "++show n
--  | STOREa | STOREr
--  | ALLOC | SLIDE | CALL | ENTER  | RETURN
--  | MALLOC | NEW
--  | MARK
showInst k HALT = "HALT"

In [28]:
c = [LOADc 1, JUMP c]

In [29]:
c
-- <code> 안에 있는 내용을 까보고 싶으면 show에서 showInst 1로 한 레벨 더 출력

-- 또는 직접 JUMP 인스트럭션을 뽑아와서 패턴 매칭
JUMP c' = c!!1

c'

[LOADc 1,JUMP <code>]

[LOADc 1,JUMP <code>]

In [30]:
import Data.Maybe (fromJust)

:type fromJust

fromJust (Just 4)

fromJust Nothing

fromJust :: forall a. Maybe a -> a

4

Maybe.fromJust: Nothing

In [31]:
lookup' x = fromJust . lookup x

:type lookup
:type lookup'

lookup :: forall a b. Eq a => a -> [(a, b)] -> Maybe b

lookup' :: forall a c. Eq a => a -> [(a, c)] -> c

In [32]:
import qualified Data.Map as Map
import Data.Map (Map, (!)) -- Data.Map 모듈에서 Map 타입과 !연산자만

type Mem = Map Addr Int

emptyMem :: Mem
emptyMem = Map.empty

In [33]:
type Stack = [Int]
type VM = (Code, Stack, Mem)

b2i False = 0
b2i True  = 1

i2b = (/= 0)

runVM :: VM -> VM
runVM vm@([], _, _) = vm
runVM vm = runVM (stepVM vm)

stepVM :: VM -> VM
stepVM (ADD:code, x:y:stack , mem) = (code, y + x : stack, mem)
stepVM (SUB:code, x:y:stack , mem) = (code, y - x : stack, mem)
stepVM (MUL:code, x:y:stack , mem) = (code, y * x : stack, mem)
stepVM (DIV:code, x:y:stack , mem) = (code, y `div` x : stack, mem)
stepVM (NEG:code, x:stack, mem) = (code, -x : stack , mem)
stepVM (AND:code, x:y:stack , mem) = (code, b2i(i2b y && i2b x) : stack, mem)
stepVM (OR:code,  x:y:stack , mem) = (code, b2i(i2b y || i2b x) : stack, mem)
stepVM (NOT:code, x:stack , mem) = (code, b2i(not $ i2b x) : stack, mem)
stepVM (EQU:code, x:y:stack , mem) = (code, b2i(x==y) : stack, mem)
stepVM (NEQ:code, x:y:stack , mem) = (code, b2i(x/=y) : stack, mem)
stepVM (GR:code,  x:y:stack , mem) = (code, b2i(y> x) : stack, mem)
stepVM (GEQ:code, x:y:stack , mem) = (code, b2i(y>=x) : stack, mem)
stepVM (LE:code,  x:y:stack , mem) = (code, b2i(y< x) : stack, mem)
stepVM (LEQ:code, x:y:stack , mem) = (code, b2i(y<=x) : stack, mem)
stepVM (DUP:code, x:stack, mem) = (code, x:x:stack, mem)
stepVM (POP:code, x:stack,  mem) = (code, stack, mem)
stepVM (JUMP c :code, stack, mem) = (c, stack, mem)
stepVM (JUMPz c :code, 0:stack, mem) = (c, stack, mem)
stepVM (JUMPz c :code, _:stack, mem) = (code, stack, mem)
stepVM (LOAD :code, addr:stack, mem) = (code, mem!addr : stack, mem)
stepVM (LOADc val :code, stack, mem) = (code, val:stack, mem)
--  | LOADa | LOADr | LOADrc
stepVM (STORE:code, addr:v:stack, mem) = (code, v:stack, Map.insert addr v mem)
stepVM (STOREc v :code, addr:stack, mem) = (code, v:stack, Map.insert addr v mem)
--  | STOREa | STOREr
--  | ALLOC | SLIDE | CALL | ENTER  | RETURN
--  | MALLOC | NEW
--  | MARK
stepVM (HALT:_, stack, mem) = ([], stack, mem)
stepVM vm = error $ "stemVM "++show vm

In [55]:
{-
codeR :: Exp -> AEnv -> Code
codeR (Lit q) rho = [LOADc q]
codeR (Var x) rho = codeL (Var x) rho ++ [LOAD]
codeR (And e1 e2) rho = codeR e1 rho ++ codeR e2 rho ++ [AND]
codeR (Or e1 e2)  rho = codeR e1 rho ++ codeR e2 rho ++ [OR]
codeR (Not e) rho = codeR e rho ++ [NOT]
codeR (Equ e1 e2) rho = undefined
codeR (Neq e1 e2) rho = undefined
codeR (Gr e1 e2) rho = undefined
codeR (Geq e1 e2) rho = undefined
codeR (Le e1 e2) rho = undefined
codeR (Leq e1 e2) rho = undefined
codeR (Neg e) rho = undefined
codeR (Add e1 e2) rho = codeR e1 rho ++ codeR e2 rho ++ [ADD]
codeR (Sub e1 e2) rho = undefined
codeR (Mul e1 e2) rho = undefined
codeR (Div e1 e2) rho = undefined
codeR (Assign (Var x) e) rho = codeR e rho ++ codeL (Var x) rho ++ [STORE]

codeL (Var x) rho = [LOADc xAddr]  where xAddr = lookup' x rho

code (Sexp e) rho = codeR e rho ++ [POP]
code (Sseq (s:ss)) rho = code s rho ++ code (Sseq ss) rho
code (Sseq []) rho = []
...

-- codeL :: Exp -> AEnv -> Code
-- codeR :: Exp -> AEnv -> Code
-- code :: Stmt -> AEnv -> Code
-}

codeL :: Exp -> AEnv -> (Code -> Code)
codeR :: Exp -> AEnv -> (Code -> Code)
code :: Stmt -> AEnv -> (Code -> Code)

codeR (Lit q)        rho = (LOADc q :) -- (\k -> LOADc q : k)
codeR (Var x)        rho = codeL (Var x) rho . (LOAD :)
codeR (And e1 e2)    rho = codeR e1 rho . codeR e2 rho . (AND :)
codeR (Or e1 e2)     rho = codeR e1 rho . codeR e2 rho . (OR :)
codeR (Not e)        rho = codeR e rho . (NOT :)
codeR (Equ e1 e2)    rho = undefined
codeR (Neq e1 e2)    rho = undefined
codeR (Gr e1 e2)     rho = undefined
codeR (Geq e1 e2)    rho = undefined
codeR (Le e1 e2)     rho = codeR e1 rho . codeR e2 rho . (LE :)
codeR (Leq e1 e2)    rho = undefined
codeR (Neg e)        rho = codeR e rho . (NEG :)
codeR (Add e1 e2)    rho = codeR e1 rho . codeR e2 rho . (ADD :)
codeR (Sub e1 e2)    rho = undefined
codeR (Mul e1 e2)    rho = undefined
codeR (Div e1 e2)    rho = undefined
codeR (Assign (Var x) e) rho = codeR e rho . codeL (Var x) rho . (STORE :)

codeL (Var x) rho = (LOADc xAddr :)  where xAddr = lookup' x rho

code (Sexp e) rho = codeR e rho . (POP :)
code (Sseq (s:ss)) rho = code s rho . code (Sseq ss) rho
code (Sseq [])     rho = \k -> k
code (If e s)         rho =  \k -> codeR e rho . (JUMPz k :) . code s rho $ k
code (IfElse e s1 s2) rho = undefined
-- code (Switch e ss d) = undefined
code (While e s) rho = \k ->
  let c = codeR e rho . (JUMPz k :) . code s rho . (JUMP c :) $ k
   in c
code (For (e1,e2,e3) s) rho = undefined

In [56]:
c1 = codeR (Add (Lit 3) (Lit 4)) []
:type c1

c1 :: Code -> Code

In [57]:
c1 []
c1 [LOADc 2, MUL]

[LOADc 3,LOADc 4,ADD]

[LOADc 3,LOADc 4,ADD,LOADc 2,MUL]

In [58]:
runVM (c1 [], [], emptyMem)

([],[7],fromList [])

In [59]:
runVM ([LOADc 2, MUL], [7], emptyMem)

([],[14],fromList [])

In [60]:
runVM (c1 [LOADc 2, MUL], [], emptyMem)

([],[14],fromList [])

In [68]:
-- x의 절대값을 구해서 y에 저장하자.
rho = [("x",1),("y",2)] -- x를 주소 1, y를 주소 2의 메모리 공간에 할당

{-
   { x <- ?
     y <- x;
    if (y<0) y <- -y; }
-}
absExpr = Sseq [ Sexp(Assign (Var "x") (Lit (-3))),
                 Sexp(Assign (Var "y") (Var "x")),
                 If (Le(Var "y")(Lit 0)) (Sexp(Assign (Var "y") (Neg(Var "y"))))]

c = code absExpr rho $ [HALT]
c

runVM (c,[],emptyMem)

[LOADc -3,LOADc 1,STORE,POP,LOADc 1,LOAD,LOADc 2,STORE,POP,LOADc 2,LOAD,LOADc 0,LE,JUMPz <code>,LOADc 2,LOAD,NEG,LOADc 2,STORE,POP,HALT]

([],[],fromList [(1,-3),(2,3)])

# hw2 명령형 언어 컴파일러 작성 과제 (6점)
* `codeR`과 `code` 함수를 완성하라 (2점)
* 팩토리알 프로그램을 `For`와 `IfElse`문을 사용해 작성하고 `code`함수로 컴파일하여 `runVM`으로 실행해 보라 (2점; 아직 변수 정의를 컴파일하는 코드는 작성하지 않았으므로 필요한 각각의 변수와 그에 대응되는 메모리 주소를 AEnv 환경에 직접 제공하는 방식으로 )
* 팩토리알 프로그램에서 사용되지 않은 나머지 새로 작성한 각각의 문법요소이 대해 `codeR` 또는 `code`를 이용한 컴파일하고 `runVM`을 사용해 실행한 테스트 케이스를 작성하라 (2점; 하나라도 빠지면 1점 감점, 절반 이하 수준이면 0점)